BPMN Workflow (Microservices)
----------------------------------------------

![](images/Microservices-BPMN.png)

- - -

Die [Business Process Model and Notation](https://de.wikipedia.org/wiki/Business_Process_Model_and_Notation) (BPMN, deutsch Geschäftsprozessmodell und -notation) ist eine grafische Spezifikationssprache in der Wirtschaftsinformatik und im Prozessmanagement. Sie stellt Symbole zur Verfügung, mit denen Fach-, Methoden- und Informatikspezialisten Geschäftsprozesse und Arbeitsabläufe modellieren und dokumentieren können.

[Camunda BPM](https://camunda.com/de/) ist ein in Java geschriebenes freies Workflow-Management-System, mit dem Geschäftsprozesse in BPMN 2.0 definiert und ausgeführt werden können.

Das nachfolgende BPMN Beispiel basiert auf dem Blog Eintrag [Use Camunda without touching Java and get an easy-to-use REST-based orchestration and workflow engine](https://blog.bernd-ruecker.com/use-camunda-without-touching-java-and-get-an-easy-to-use-rest-based-orchestration-and-workflow-7bdf25ac198e) von Bernd Rücker.

- - -
Das Beispiel besteht aus folgenen Microservices

* Einer BPMN **Workflow Engine** (Camunda) mit einen Prozess zur Freigabe von Rechnungen.
* Ein **Frontend** in HTML und JS implementiert zum starten des obigen Prozesses.
* Ein **Backend** zum Verbuchen der Rechnungen implementiert in Java.

In [ ]:
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/bpmn/bpmn-backend.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/bpmn/bpmn-frontend.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/bpmn/camunda.yaml

- - - 
Nach dem Starten der Microservices müssen wir den BPMN Prozess in der Workflow Engine veröffentlichen.

In [ ]:
! wget https://raw.githubusercontent.com/mc-b/bpmn-tutorial/master/RechnungStep3.bpmn
! curl -w "\n" -H "Accept: application/json" -F "deployment-name=rechnung" -F "enable-duplicate-filtering=true" -F "deploy-changed-only=true" -F "RechnungStep3.bpmn=@RechnungStep3.bpmn" http://camunda:8080/engine-rest/deployment/create    

- - -
Nun können wir neue Rechnungen über das Frontend erfassen und diese in der Workflow Engine anschauen. Die URLs sind wie folgt:

In [ ]:
! echo "BPMN Workflow Engine: http://"$(cat ~/work/server-ip)/camunda # User/Password: demo"
! echo "Frontend            : http://"$(cat ~/work/server-ip)/frontend/

Im `Cockpit` sollte unter `Process Definitions` ein Prozess `RechnungStep3` ersichtlich sein. Fehlt dieser, den Befehl oben für die Veröffentlichung des Prozesses nochmals ausführen.

Mittels dem Frontend oder nachfolgenden Befehl können wir diesen Prozess starten. Der gestartete Prozess ist dann unter `Tasklist` ersichtlich.


In [ ]:
%%bash
curl \
-H "Content-Type: application/json" \
-X POST \
-d '{ "variables": { "rnr": {"value": "123", "type": "long"}, "rbetrag": {"value": "200.00", "type": "String"} } }' \
http://camunda:8080/engine-rest/process-definition/key/RechnungStep3/start

- - -
Aufräumen

In [ ]:
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/bpmn/bpmn-backend.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/bpmn/bpmn-frontend.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/bpmn/camunda.yaml